# Liabraries

In [ ]:
from ast                                import literal_eval
from IPython.display                    import clear_output
from bs4                                import BeautifulSoup as bs
from selenium                           import webdriver
from selenium.webdriver.common.by       import By
from selenium.webdriver.chrome.options  import Options
from selenium.webdriver.support.ui      import WebDriverWait
from selenium.common.exceptions         import TimeoutException
from webdriver_manager.chrome           import ChromeDriverManager
from selenium.webdriver.support         import expected_conditions as EC
import pandas as pd, requests as rq, time, os, sys, logging, re

# Functions

In [ ]:
def chrome_browser():
    chrome_options = Options()
    chrome_prefs = {}
    chrome_prefs["profile.default_content_settings"] = {"images": 2}
    chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}
    chrome_options.add_argument("start-maximized")
    chrome_options.add_experimental_option("prefs", chrome_prefs)
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    return driver

def Insta_login(username,password):
    user_name = driver.find_element(By.CSS_SELECTOR, '._ab32:nth-child(1) ._ac4d')
    user_name.send_keys(username)
    user_password = driver.find_element(By.CSS_SELECTOR, '._ab32+ ._ab32 ._ac4d')
    user_password.send_keys(password)
    login_button = driver.find_element(By.CSS_SELECTOR, '._acap')
    login_button.submit()
    print(f'Your are succefully Login using this {username} Username')
    return

In [ ]:
driver = chrome_browser()
driver.get('https://www.instagram.com/')

# Insta Login

In [ ]:
username = '******'
password = '******'
time.sleep(2)
Insta_login(username,password)

# Username Lists

In [ ]:
# Read the Excel file containing the Instagram usernames
excel_file_path = r"C:\Users\hp\Music\Insta_user_scraper\following_users.xlsx"
df = pd.read_excel(excel_file_path)
df.drop_duplicates(inplace = True)

In [ ]:
new_df = pd.DataFrame(columns=['Username', 'First Name', 'Last Name', 'Following Count', 'Followers Count', 'Private', 'Bio', 'Email', 'Total_post', 'Profile URLs'])

In [ ]:
# Regex pattern to match an email address in bio
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

# Create an empty list to store profile URLs
profile_urls = []
# Create an empty list to store DataFrames
df_list = []

# Get the total count of usernames
total_count = len(df['following_username'].to_list())
new_df['Error Links'] = ''

for count, username in enumerate(df['following_username'].to_list(), start=1):
    profile_url = f'https://www.instagram.com/{username}/'
    driver.get(profile_url)

    try:
        # Wait until the element is present
        element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '.x1ez9qw7 img'))
        profile_img = WebDriverWait(driver, 10).until(element_present)
        # Get the src attribute of the image element
        profile_img_url = profile_img.get_attribute('src')

        # Wait until the element is visible
        element_visible = EC.visibility_of_element_located((By.CSS_SELECTOR, '.x1ez9qw7'))
        details_element = WebDriverWait(driver, 10).until(element_visible)

        # Get the text value of the element
        details_text = details_element.text

        # Split the text into separate values
        details_list = details_text.split('\n')

        # Extract full name and split into first name and last name
        name_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '._aad7')))
        name_list = name_element.text
        first_name = name_list.split()[0]
        last_name = ' '.join(name_list.split()[1:])

        # Extract the individual values
        username = details_list[0]
        edit_profile = details_list[1]
        posts = details_list[3].split()[0]

        followers = None
        following = None

        for detail in details_list:
            if 'followers' in detail:
                followers = detail.split()[0]
            elif 'following' in detail:
                following = detail.split()[0]

            # Break the loop if both followers and following are found
            if followers and following:
                break

        bio = ' '.join(details_list[6:])

        # Extracting email from bio if present
        email = re.search(email_pattern, bio)
        if email:
            email = email.group()

        # Extracting profile URL
        profile_url = driver.current_url

        # Append the URL to the profile_urls list if it is not already present
        if profile_url not in profile_urls:
            profile_urls.append(profile_url)

        # Append the details to the new DataFrame
        new_row = {
            'Username': username,
            'First Name': first_name,
            'Last Name': last_name,
            'Following Count': following,
            'Followers Count': followers,
            'Post Count': posts,
            'Private': False,
            'User Bio': bio,
            'Email': email,
            'Profile URLs': profile_urls[0],
            'Profile Pic Url': profile_img_url
        }
        df_list.append(pd.DataFrame(new_row, index=[0]))

    except Exception as e:
        # Save the error URL in the 'Error Link' column
        new_df.loc[count - 1, 'Error Link'] = profile_url
        print(f"Error processing profile URL: {profile_url}")
        continue

    # Refresh the count
    print(f"Processed {count}/{total_count} usernames.", end='\r')

# Concatenate the DataFrame list into a single DataFrame
new_df = pd.concat(df_list, ignore_index=True)

# Print completion message
print("Extraction completed.")

In [ ]:
df_combined = pd.concat(df_list, ignore_index=True)

In [ ]:
df_combined.to_excel('Insta_Users_Data.xlsx', index=False)